In [24]:
# imports necessários:
import pandas as pd
from pathlib import PosixPath 
from datetime import datetime
import sqlite3 as sql

#Endereço dos arquivos CSV:
local_arquivos = PosixPath('/')


# Coleta dos dados: IBGE

In [25]:
#Coletar dados dos municípios do Rio de Janeiro (site IBGE)
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

#Transformar os dados em um Data Frame
ibge_dados_rj = pd.DataFrame(pd.read_html(url, match='Municípios do Rio de Janeiro')[0])

#Ajustar a nomenclatura das colunas do Data Frame:
ibge_dados_rj = ibge_dados_rj.rename(columns = {'Municípios do Rio de Janeiro': 'nomeMunic', 'Códigos': 'codMunic'})

#Adicionar coluna de tempo ao Data Frame:
ibge_dados_rj['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

#Ajustar index do Data Frame para idMunic
ibge_dados_rj.index.name = 'idMunic'
#Ajustar numeração do índice, para começar em 1
ibge_dados_rj.index = ibge_dados_rj.index + 1


# Coleta dos dados: Delegacias

### Cadastro das Delegacias de Polícia RJ

In [26]:
#Criar Data Frame com os dados das DPs
tb_log_dp = pd.DataFrame(pd.read_csv(local_arquivos / "DP.csv"))

#Ajustar a nomenclatura das colunas do Data Frame:
tb_log_dp = tb_log_dp.rename(columns = {'COD_DP': 'idDP', 'NM_DP':'nomeDP', 'Endereço':'enderDP'})

#Adicionar coluna de Data e Hora da carga no Data Frame 
tb_log_dp['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')


### Cadastro dos Responsáveis pelas Delegacias de Polícia RJ


In [44]:
#Criar Data Frame com os dados dos responsáveis pelas DPs
tb_log_resp_dp = pd.DataFrame(pd.read_csv(local_arquivos / "ResponsavelDP.csv"))
                                     
#Ajustar a nomenclatura das colunas do Data Frame:
tb_log_resp_dp = tb_log_resp_dp.rename(columns = {'COD_DP': 'idDP', 'Responsavel':'nomeResp'})

#Adicionar coluna de Data e Hora da carga no Data Frame 
tb_log_resp_dp['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')


# Coleta dos dados: Batalhões Polícia Militar


### Cadastro dos registros dos batalhões


In [47]:
#Criar Data Frame com os dados dos BPM
tb_log_bpm = pd.DataFrame(pd.read_csv(local_arquivos / "BPM.csv"))
tb_log_bpm['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

#Ajustar a nomenclatura das colunas do Data Frame:
tb_log_bpm = tb_log_bpm.rename(columns = {'COD_BPM': 'idBPM', 'NM_BPM':'nomeBPM','Endereco':'enderBPM'})


,idBPM,nomeBPM,enderBPM,dtCarga
0,2,2º BPM,"R. Álvaro Ramos, 155",19/04/2022 17:58
1,3,3º BPM,"R. Lucídio Lago, 181",19/04/2022 17:58
2,4,4º BPM,"R. Francisco Eugênio, 228",19/04/2022 17:58
3,5,5º BPM,"Praça Cel. Assunção, S/N",19/04/2022 17:58
4,6,6º BPM,"R. Barão de Mesquita, 625",19/04/2022 17:58
5,7,7º BPM,"R. Dr. Alfredo Backer, 367",19/04/2022 17:58
6,8,8º BPM,"Rua Tenente Coronel Cardoso, s/n",19/04/2022 17:58
7,9,9º BPM,"R. Tacaratu, 94",19/04/2022 17:58
8,10,10º BPM,"Rod. Lúcio Meira, 47000",19/04/2022 17:58
9,11,11º BPM,"R. Voluntários da Pátria, 474",19/04/2022 17:58


# Configuração Banco de Dados: ODS e DW - SQLite

In [29]:
##Manipular sistema de arquivos:
endereco = PosixPath('/')

#Local onde serão salvas as bases de dados
bd_ods = endereco / "ibgeODS.db"
bd_dw = endereco / "ibgeDW.db"

#Validar a existência das bases
if endereco.exists():
    if (bd_ods.exists() and bd_dw.exists()):
        print('Bancos de dados já existem!')
    else:
        bd_ods.touch()
        bd_dw.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe, favor verificar!')
    


Bancos de dados já existem!


### Criação das tabelas - ODS

In [54]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Definir variável para manipulação
cursor_ods = conexao_ods.cursor()

#Criar tabelase e índices, caso não existam:
####---------------tbLogMunic---------------####
cursor_ods.execute(

    '''
        CREATE TABLE IF NOT EXISTS "tbLogMunic" (
          "idMunic" INTEGER,
          "nomeMunic" TEXT,
          "codMunic" INTEGER,
          "dtCarga" TEXT
        )
     '''
)
cursor_ods.execute('CREATE INDEX IF NOT EXISTS idx_tbLogMunic_idMunic ON tbLogMunic(idMunic)')
####----------------------------------------####
####---------------tbLogDP------------------####
cursor_ods.execute(

    '''
        CREATE TABLE IF NOT EXISTS "tbLogDP" (
          "idDP" INTEGER,
          "nomeDP" VARCHAR(200),
          "enderDP" VARCHAR(200),
          "dtCarga" DATETIME
        )
        
    '''
)
cursor_ods.execute('CREATE INDEX IF NOT EXISTS idx_tbLogDP_idDP ON tbLogDP(idDP)')
####----------------------------------------####
####---------------tbLogRespDP--------------####
cursor_ods.execute(

    '''
        CREATE TABLE IF NOT EXISTS "tbLogRespDP" (
          "idDP" INTEGER,
          "nomeResp" VARCHAR(200),
          "dtCarga" DATETIME
        )
     '''
)
cursor_ods.execute('CREATE INDEX IF NOT EXISTS idx_tbLogRespDP_idDP ON tbLogRespDP(idDP)')
####----------------------------------------####
####---------------tbLogBPM-----------------####
cursor_ods.execute(

    '''
        CREATE TABLE IF NOT EXISTS "tbLogBPM" (
          "idBPM" INTEGER,
          "nomeBPM" VARCHAR(50),
          "enderBPM" VARCHAR(200),
          "dtCarga" DATETIME
        )
        
    '''
)
cursor_ods.execute('CREATE INDEX IF NOT EXISTS idx_tbLogBPM_idBPM ON tbLogBPM(idBPM)')
####---------------------------------------####

#Confirmar insert e encerrando a conexão
conexao_ods.commit()
conexao_ods.close()

### Criação das tabelas - DW

In [56]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Definir variável para manipulação
cursor_dw = conexao_dw.cursor()

#Criar tabelase índices, caso não existam:
####---------------dMunicipio---------------####
cursor_dw.execute(

    '''
        CREATE TABLE IF NOT EXISTS "dMunicipio" (
          "idMunic" INTEGER,
          "codMunic" INTEGER,
          "nomeMunic" TEXT
        )

     '''
)
cursor_dw.execute('CREATE INDEX IF NOT EXISTS idx_dMunicipio_idMunic ON dMunicipio(idMunic)')
####---------------------------------------####
####------------------dDP------------------####
cursor_dw.execute(

    '''
        CREATE TABLE IF NOT EXISTS "dDP" (
          "idDP"  INTEGER PRIMARY KEY AUTOINCREMENT ,
          "nomeDP" VARCHAR(200),
          "enderDP" VARCHAR(200),
          "nomeResp" VARCHAR(200)
        )

     '''
)
cursor_dw.execute('CREATE INDEX IF NOT EXISTS idx_dDP_idDP ON dDP(idDP)')
####---------------------------------------####
#Confirmar create e encerrar a conexão
conexao_dw.commit()
conexao_dw.close()

# Manipulação dos Dados - ODS

### Inserir os registros ODS: Cadastro Municípios

In [30]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Definir variável para manipulação
cursor_ods = conexao_ods.cursor()

#Adicionar novos registros à tabela tbLogMunic
ibge_dados_rj.to_sql('tbLogMunic', conexao_ods, if_exists='append')

#Confirmar insert e encerrando a conexão
conexao_ods.commit()
conexao_ods.close()

print("Carga ODS Cadastro de Municípios finalizada!",len(ibge_dados_rj),"Registros inseridos!")

Carga ODS Cadastro de Municípios finalizada! 92 Registros inseridos!


### Inserir os registros ODS: Cadastro DPs

In [31]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Definir variável para manipulação
cursor_ods = conexao_ods.cursor()

#Inserir dados
cursor_ods.executemany('''INSERT INTO tbLogDP(idDP,nomeDP,enderDP,dtCarga) VALUES(?,?,?,?)''',tb_log_dp.values.tolist())

#Confirmar insert e encerrando a conexão
conexao_ods.commit()
conexao_ods.close()

#Mensagem de conslusão
print("Carga ODS Cadastro de DPs finalizada!",len(tb_log_dp),"Registros inseridos!")


Carga ODS Cadastro de DPs finalizada! 137 Registros inseridos!


### Inserir os registros ODS: Cadastro Responsáveis DPs

In [33]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Definir variável para manipulação
cursor_ods = conexao_ods.cursor()

#Inserir dados
cursor_ods.executemany('''INSERT INTO tbLogRespDP(idDP,nomeResp,dtCarga) VALUES(?,?,?)''',tb_log_resp_dp.values.tolist())

#Confirmar insert e encerrando a conexão
conexao_ods.commit()
conexao_ods.close()

#Mensagem de conslusão
print("Carga ODS Responsáveis de DPs finalizada!",len(tb_log_resp_dp),"Registros inseridos!")

Carga ODS Responsáveis de DPs finalizada! 137 Registros inseridos!


### Inserir os registros ODS: Cadastro BPM

In [50]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Definir variável para manipulação
cursor_ods = conexao_ods.cursor()

#Inserir dados
cursor_ods.executemany('''INSERT INTO tbLogBPM(idBPM,nomeBPM,enderBPM,dtCarga) VALUES(?,?,?,?)''',tb_log_bpm.values.tolist())

#Confirmar insert e encerrando a conexão
conexao_ods.commit()
conexao_ods.close()

#Mensagem de conslusão
print("Carga ODS Cadastro de DPs finalizada!",len(tb_log_bpm),"Registros inseridos!")

Carga ODS Cadastro de DPs finalizada! 39 Registros inseridos!


### Validar os registros inseridos ODS - Cadastro DPs

In [32]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Select de validação:
select_ods = 'SELECT * FROM tbLogDP ORDER BY dtCarga DESC LIMIT '+ str(len(tb_log_dp))
select_ods = pd.read_sql(select_ods,conexao_ods)

#Encerrar a conexão
conexao_ods.close()

#Exibir select de validação
select_ods

,idDP,nomeDP,enderDP,dtCarga
0,1,001ª DP - Praça Mauá,Atendendo provisoriamente na sede da 4ª DP - P...,19/04/2022 17:45
1,4,004ª DP - Praça da República,"Av. Presidente Vargas, 1100 - Centro, Rio de J...",19/04/2022 17:45
2,5,005ª DP - Mem de Sá,"Avenida Gomes Freire, 320 - Centro, Rio de Jan...",19/04/2022 17:45
3,6,006ª DP - Cidade Nova,"Rua Professor Clementino Fraga, 77 - Centro, R...",19/04/2022 17:45
4,7,007ª DP - Santa Teresa,"Rua Francisco de Castro, 5 - Santa Teresa, Rio...",19/04/2022 17:45
...,...,...,...,...
132,159,159ª DP - Cachoeira de Macacu,"Avenida Lord Baden Powel, 93 - Centro, Cachoei...",19/04/2022 17:45
133,165,165ª DP - Mangaratiba,"Estrada São João Marcos, s/n - Praia do Saco, ...",19/04/2022 17:45
134,166,166ª DP - Angra dos Reis,"Rua Doutor Coutinho, 6 - Centro, Angra dos Rei...",19/04/2022 17:45
135,167,167ª DP - Paraty,"Rua Roberto da Silveira , s/n - Vila Colonial,...",19/04/2022 17:45


### Validar os registros inseridos ODS - Cadastro Responsáveis DPs

In [34]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Select de validação:
select_ods = 'SELECT * FROM tbLogRespDP  ORDER BY dtCarga DESC LIMIT ' + str(len(tb_log_resp_dp))
select_ods = pd.read_sql(select_ods,conexao_ods)

#Encerrar a conexão
conexao_ods.close()

#Exibir select de validação
select_ods

,idDP,nomeResp,dtCarga
0,1,Delegado de Polícia José Luiz Silva Duarte,19/04/2022 17:45
1,4,Delegada de Polícia Patricia de Paiva Aguiar,19/04/2022 17:45
2,5,Delegado de Polícia Bruno Gilaberte Freitas,19/04/2022 17:45
3,6,Delegado de Polícia Fabio Luiz Da Silva Souza,19/04/2022 17:45
4,7,Delegado de Polícia Carlos Alberto Meirelles D...,19/04/2022 17:45
...,...,...,...
132,159,Delegado de Polícia Clovis Souza Moreira,19/04/2022 17:45
133,165,Delegado de Polícia Anderson Ribeiro Pinto,19/04/2022 17:45
134,166,Delegado de Polícia Vilson de Almeida Silva,19/04/2022 17:45
135,167,Delegado de Polícia Marcelo dos Santos Haddad,19/04/2022 17:45


### Validar os registros inseridos ODS - Cadastro BPM

In [52]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Select de validação:
select_ods = 'SELECT * FROM tbLogBPM  ORDER BY dtCarga DESC LIMIT ' + str(len(tb_log_bpm))
select_ods = pd.read_sql(select_ods,conexao_ods)

#Encerrar a conexão
conexao_ods.close()

#Exibir select de validação
select_ods

,idBPM,nomeBPM,enderBPM,dtCarga
0,2,2º BPM,"R. Álvaro Ramos, 155",19/04/2022 17:58
1,3,3º BPM,"R. Lucídio Lago, 181",19/04/2022 17:58
2,4,4º BPM,"R. Francisco Eugênio, 228",19/04/2022 17:58
3,5,5º BPM,"Praça Cel. Assunção, S/N",19/04/2022 17:58
4,6,6º BPM,"R. Barão de Mesquita, 625",19/04/2022 17:58
5,7,7º BPM,"R. Dr. Alfredo Backer, 367",19/04/2022 17:58
6,8,8º BPM,"Rua Tenente Coronel Cardoso, s/n",19/04/2022 17:58
7,9,9º BPM,"R. Tacaratu, 94",19/04/2022 17:58
8,10,10º BPM,"Rod. Lúcio Meira, 47000",19/04/2022 17:58
9,11,11º BPM,"R. Voluntários da Pátria, 474",19/04/2022 17:58


# Manipulação dos Dados - DW

### Inserir os registros: Cadastro de Municípios

In [35]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Definir variável para manipulação
cursor_dw = conexao_dw.cursor()

#Criar tabela, caso não exista:
cursor_dw.execute(

    '''
        CREATE TABLE IF NOT EXISTS "dMunicipio" (
          "idMunic" INTEGER,
          "codMunic" INTEGER,
          "nomeMunic" TEXT
        )

     '''
)

#Criar Data Frame para popular o DW
ibge_dados_rj_dw = ibge_dados_rj[['codMunic','nomeMunic']]

#Criar e/ou popular a dimensão dMunicipio com o DF ibge_dados_rj_dw
ibge_dados_rj_dw.to_sql('dMunicipio', conexao_dw, if_exists='replace')

#Confirmar insert e encerrando a conexão
conexao_dw.commit()
conexao_dw.close()

#Mensagem de conslusão
print("Carga ODS Responsáveis de DPs finalizada!",len(ibge_dados_rj_dw),"Registros inseridos!")

Carga ODS Responsáveis de DPs finalizada! 92 Registros inseridos!


### Inserir os registros: DPs e responsáveis

In [37]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Definir variável para manipulação
cursor_dw = conexao_dw.cursor()

#Query para dimensão dDP:

dDP ='''
SELECT 
    [Identificador do Departamento],
    [Nome do Departamento],
    [Endereço do Departamento],
    [Nome do Responsável]
FROM
    (
        SELECT 
            DP.idDP [Identificador do Departamento],
            DP.nomeDP [Nome do Departamento],
            DP.enderDP [Endereço do Departamento],
            RESP.nomeResp [Nome do Responsável],
            MAX(DP.dtCarga) [Horário da última carga dos dados]
        FROM tbLogDP DP
            INNER JOIN tbLogRespDP RESP on RESP.idDP = DP.idDP
        GROUP BY
            DP.idDP,
            DP.nomeDP,
            DP.enderDP,
            RESP.nomeResp
    ) dDP
'''
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Preencher o Data Frame da dimensão dDP
dDP = pd.read_sql(dDP,conexao_ods)

#Confirmar a leitura e encerrando a conexão ODS
conexao_ods.commit()
conexao_ods.close()

#Limpar a dimensão para inserir os novos registros:
cursor_dw.execute(''' DELETE FROM dDP''')
cursor_dw.execute('UPDATE sqlite_sequence SET seq=0 WHERE name="dDP"')

#Inserir os registros
cursor_dw.executemany('''INSERT INTO dDP(idDP,nomeDP,enderDP,nomeResp) VALUES(?,?,?,?)''',dDP.values.tolist())

#Confirmar o delete, insert e encerrando a conexão DW
conexao_dw.commit()
conexao_dw.close()

### Validar os registros inseridos DW - Dimensão Municípios

In [36]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Select de validação:
select_dw = pd.read_sql('SELECT * FROM dMunicipio',conexao_dw)

#Encerrando a conexão
conexao_dw.close()

#Exibindo select de validação
select_dw

,idMunic,codMunic,nomeMunic
0,1,3300100,Angra dos Reis
1,2,3300159,Aperibé
2,3,3300209,Araruama
3,4,3300225,Areal
4,5,3300233,Armação dos Búzios
...,...,...,...
87,88,3306008,Três Rios
88,89,3306107,Valença
89,90,3306156,Varre-Sai
90,91,3306206,Vassouras


### Validar os registros inseridos DW - Dimensão DPs

In [38]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Select de validação:
select_dw = pd.read_sql('''SELECT 
                                idDP [Identificador do Departamento],
                                nomeDP [Nome do Departamento],
                                enderDP [Endereço do Departamento],
                                nomeResp [Nome do Responsável] 
                           FROM dDP'''
            ,conexao_dw)

#Encerrar a conexão
conexao_dw.close()

#Exibir select de validação
select_dw

,Identificador do Departamento,Nome do Departamento,Endereço do Departamento,Nome do Responsável
0,1,001ª DP - Praça Mauá,Atendendo provisoriamente na sede da 4ª DP - P...,Delegado de Polícia José Luiz Silva Duarte
1,4,004ª DP - Praça da República,"Av. Presidente Vargas, 1100 - Centro, Rio de J...",Delegada de Polícia Patricia de Paiva Aguiar
2,5,005ª DP - Mem de Sá,"Avenida Gomes Freire, 320 - Centro, Rio de Jan...",Delegado de Polícia Bruno Gilaberte Freitas
3,6,006ª DP - Cidade Nova,"Rua Professor Clementino Fraga, 77 - Centro, R...",Delegado de Polícia Fabio Luiz Da Silva Souza
4,7,007ª DP - Santa Teresa,"Rua Francisco de Castro, 5 - Santa Teresa, Rio...",Delegado de Polícia Carlos Alberto Meirelles D...
...,...,...,...,...
132,159,159ª DP - Cachoeira de Macacu,"Avenida Lord Baden Powel, 93 - Centro, Cachoei...",Delegado de Polícia Clovis Souza Moreira
133,165,165ª DP - Mangaratiba,"Estrada São João Marcos, s/n - Praia do Saco, ...",Delegado de Polícia Anderson Ribeiro Pinto
134,166,166ª DP - Angra dos Reis,"Rua Doutor Coutinho, 6 - Centro, Angra dos Rei...",Delegado de Polícia Vilson de Almeida Silva
135,167,167ª DP - Paraty,"Rua Roberto da Silveira , s/n - Vila Colonial,...",Delegado de Polícia Marcelo dos Santos Haddad
